# Data Science Capstone Project

## Introduction
The common saying in real estate is the three most important things when buying property are location, location, location. In my capstone project I will attempt to quantify the impact that the venues in a given location have on the price of both buying and short-term renting there.

There were a few different lenses I wanted to use to look at the effect of venues on prices:
* Total number of venues in a neighborhood
* Variety of venues
* Density of venues
* Types of venues - is there one type of venue that affects price more drastically than others?

## Data
I used three different sources to gather my data for this project:
* Foursquare
* Airbnb
* Zillow

I used the Foursquare API to gather information about the venues in each neighborhood. I downloaded summary information and metrics for all rental listings in New York City from [Airbnb](http://insideairbnb.com/get-the-data.html). I downloaded the average sale price of homes for each neighborhood in NY from [Zillow](https://www.zillow.com/research/data/).

## Methodology
### Importing packages and data

In [1]:
import pandas as pd
import numpy as np

In [142]:
buydf = pd.read_csv('NY_Zhvi_AllHomes.csv')
buydf.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1/31/2019,...,7/31/2019,8/31/2019,9/30/2019,10/31/2019,11/30/2019,12/31/2019,1/31/2020,2/29/2020,3/31/2020,4/30/2020
0,270958,3,Upper West Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,1290079,...,1245653,1231122,1218079,1211863,1214994,1223361,1223522,1218477,1207736,1208018
1,270957,5,Upper East Side,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,957125,...,943890,943323,941689,939559,938245,938958,938388,939042,932658,930498
2,194430,9,East New York,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,Kings County,477839,...,485335,485671,487120,487978,489206,490597,494343,498805,503472,507712
3,198687,11,Washington Heights,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,New York County,599515,...,586022,582360,574867,568781,564991,560923,556293,551425,546846,542495
4,272816,14,Astoria,Neighborhood,NY,NY,New York,New York-Newark-Jersey City,Queens County,648120,...,647174,647857,648439,648451,647437,648020,649250,650636,650432,651140


In [167]:
rentdf = pd.read_csv('nyc_airbnb_listings.csv')
rentdf.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2060,Modern NYC,2259,Jenny,Manhattan,Washington Heights,40.85722,-73.93790,Private room,100,1,1,2008-09-22,0.01,1,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,3,48,2019-11-04,0.37,2,335
2,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,322,2020-06-07,4.64,1,276
3,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,78,2019-10-13,0.58,1,0
4,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68688,-73.95596,Private room,60,29,50,2019-12-02,0.37,1,365


### Cleaning Data
Let's get rid of all the data/columns we don't need. 

First, we'll filter out the rows we don't need from the Zillow data. The original file contains property prices for all cities in New York state, while we're only interested in NYC.

In [144]:
buydf = buydf[buydf['City']=='New York']

Next, let's check both dataframes for any duplicate regions/neighborhoods in different county or neighborhood groups

In [145]:
regions = buydf[['RegionName']]
regions = regions[regions.duplicated(keep=False)]
regions

,RegionName
26,Murray Hill
55,Sunnyside
162,Bay Terrace
192,Murray Hill
242,Bay Terrace
354,Sunnyside


In [146]:
# Group by neighborhood group & neighborhood first since there are many listings for each
rents = rentdf.groupby(['neighbourhood_group','neighbourhood']).size().reset_index()
rents = rents[['neighbourhood']]
rents[rents.duplicated(keep=False)]

,neighbourhood


There aren't any duplicates in the rental data, but there are 3 duplicate cities in the zillow data. Since the region names and the neighborhood group names don't match up and there's no way of knowing which one is right we'll just get rid of both of them. We know the indicies of the rows from the previous regions dataframe.

In [147]:
buydf.drop(index=regions.index, inplace=True)

Alright, now let's drop the columns we don't need.

Start with Zillow data:

In [148]:
# We also only care about the Neighborhood (RegionName) and price
# For price let's average prices over the last 12 months & drop the rest
buydf.drop(columns=['RegionID', 'SizeRank', 'RegionType','StateName','State','City',
            'Metro','CountyName', '1/31/2019', '2/28/2019', '3/31/2019', '4/30/2019'], inplace=True)

In [149]:
buydf['Avg Price'] = buydf.mean(axis=1)

In [161]:
buydf = buydf[['RegionName', 'Avg Price']]
buydf.reset_index(drop=True, inplace=True)
buydf.head()

,RegionName,Avg Price
0,Upper West Side,1.226573e+06
1,Upper East Side,9.402832e+05
2,East New York,4.916105e+05
3,Washington Heights,5.680238e+05
4,Astoria,6.488646e+05


Now the airbnb data:

In [168]:
# To keep things simple, let's ignore shared rooms/homes and focus on entire units
rentdf = rentdf[rentdf['room_type'] == 'Entire home/apt']
rentdf = rentdf[['neighbourhood', 'latitude', 'longitude', 'price']]
rentdf.reset_index(drop=True, inplace=True)
rentdf.head()

,neighbourhood,latitude,longitude,price
0,Midtown,40.75362,-73.98377,225
1,Clinton Hill,40.68514,-73.95976,89
2,Murray Hill,40.74767,-73.97500,200
3,Sunset Park,40.66120,-73.99423,253
4,Chinatown,40.71344,-73.99037,150


Let's calculate the average price per night for each neighborhood

In [169]:
avgrent = rentdf.groupby('neighbourhood').mean()
avgrent.head()

,latitude,longitude,price
neighbourhood,,,
Allerton,40.865207,-73.858592,136.117647
Arden Heights,40.554263,-74.177167,118.000000
Arrochar,40.592788,-74.068481,166.333333
Arverne,40.592417,-73.794618,255.555556
Astoria,40.764084,-73.920064,139.037940


### Visualizing the data